### Look at first run results

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import imageio
import glob
import matplotlib.path as mpath
import cmocean

In [2]:
# To make circular maps:
theta = np.linspace(0, 2*np.pi, 100)
map_circle = mpath.Path(np.vstack([np.sin(theta), np.cos(theta)]).T * 0.5 + [0.5, 0.5])

In [9]:
def prepare_plot(figsize=(15, 8)):
    """This function returns prepared axes for the polar plot.
    
    Usage:
        fig, ax = prepare_plot()
    """
    fig, ax = plt.subplots(1,2, figsize=figsize, subplot_kw={'projection': ccrs.SouthPolarStereo()})
    for axis in ax.ravel():
        axis.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())
        axis.set_boundary(map_circle, transform=axis.transAxes)
        # ax.coastlines(); 
        gl = axis.gridlines(draw_labels=True, ylocs=[]);#, color='w', lw=2);
    return fig, ax

In [4]:
mesh  = xr.open_dataset('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/bathymetry/mesh_mask-20231025-filledlakes.nc').isel(time_counter=0)
fileT = xr.open_dataset('/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/new-setup-testing/run1yr/eANT025.L121_1m_19790101_19791231_grid_T.nc')

In [5]:
tmask = np.ma.masked_where(mesh.tmask.values==0, mesh.tmask.values)

In [10]:
def plot_fields(depth):

    for month in range(0,12):
        kwags1  = {'transform':ccrs.PlateCarree(), 'cmap':cmocean.cm.balance, 'rasterized':True, 'vmin':-4, 'vmax':4}
        kwags2  = {'transform':ccrs.PlateCarree(), 'cmap':cmocean.cm.haline, 'rasterized':True, 'vmin':33, 'vmax':35.5}
        fig, ax = prepare_plot()
        cm1 = ax[0].pcolormesh(fileT.nav_lon_grid_T, fileT.nav_lat_grid_T, fileT.thetao.isel(time_counter=month, deptht=depth), **kwags1)
        cm2 = ax[1].pcolormesh(fileT.nav_lon_grid_T, fileT.nav_lat_grid_T, fileT.so.isel(time_counter=month, deptht=depth), **kwags2)
        CB1 = fig.colorbar(cm1, ax=ax[0], fraction=0.04)
        CB1.set_label(fileT.thetao.long_name)
        CB2 = fig.colorbar(cm2, ax=ax[1], fraction=0.04)
        CB2.set_label(fileT.so.long_name)
        fig.suptitle(f'Month: {month+1}, depth: {mesh.nav_lev.values[depth]:.1f} m');
    
        fig.savefig(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/new-setup-testing/run1yr/m{month+1:02}z{depth:03}.jpg', bbox_inches='tight')

    return

In [11]:
def create_animation(depth):
    
    # filenames is a list of the names/locations of image files to combine into animation (mp4 in this case)
    with imageio.get_writer(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/new-setup-testing/run1yr/monthly_z{depth:03}.mp4', 
                            fps=5, mode='I') as writer: 
        for filename in filenames:  
            image = imageio.imread(filename)  
            writer.append_data(image)
    return

In [ ]:
for depth in [0, 20, 50, 80, 100]:
    plot_fields(depth)
    filenames=glob.glob(f'/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/output/new-setup-testing/run1yr/m??z{depth:03}.jpg')
    create_animation(depth)